In [1]:
import random
import csv
import os
import pickle
import sys
os.chdir('Resources/')

In [2]:
def power(b, p, m):
    b = b % m
    if p == 0:
        return 1
    j = power(b, p // 2, m)
    j = (j * j) % m
    if p % 2 == 1:
        j = (j * b) % m
    return j

In [3]:
def load_key(filename):
    with open(filename, "rb") as f:
        return pickle.load(f)

In [4]:
sys_rsa_2_public_key = load_key("17_SYS_RSA_2_Public_Key.pkl")
sys_rsa_2_public_key

(637769984173082991, 9361844235008265709)

In [5]:
input_filename = '22_(SYS)_CC_Predicted_User_Output_Data.csv'

In [6]:
def encrypt(message, public_key):
    e, n = public_key
    encrypted_message = [pow(ord(char), e, n) for char in message]
    return encrypted_message

def encrypt_csv_file(input_filename, public_key):
    with open(input_filename, 'r') as f:
        reader = csv.reader(f)
        data = "\n".join([",".join(row) for row in reader])
    
    encrypted_data = encrypt(data, public_key)
    
    return encrypted_data

encrypted_data = encrypt_csv_file(input_filename, sys_rsa_2_public_key)
print(encrypted_data, '\n', len(encrypted_data))

def decrypt(private_key, encrypted_message):
    d, n = private_key
    decrypted_message = ''.join([chr(pow(char, d, n)) for char in encrypted_message])
    return decrypted_message

su_rsa_2_private_key = load_key("16_SU_RSA_2_Private_Key.pkl")

decrypted_data = decrypt(su_rsa_2_private_key, encrypted_data)
print(decrypted_data)

[2086571625238332354, 7176964512877741619, 7822645058523022688, 8009690529124638334, 8242659079974169821, 9004561701176289127, 6500047266885745314, 7176964512877741619, 2942582851663474782, 8242659079974169821, 4955708925362059469, 2942582851663474782, 7176964512877741619, 2086571625238332354, 8242659079974169821, 2942582851663474782, 1414172889041247964, 7176964512877741619, 2086571625238332354, 8242659079974169821, 1414172889041247964, 1414172889041247964, 7176964512877741619, 4955708925362059469, 8242659079974169821, 2086571625238332354, 1414172889041247964, 7176964512877741619, 2086571625238332354, 8242659079974169821, 8009690529124638334, 1414172889041247964, 7176964512877741619, 2086571625238332354, 8242659079974169821, 1414172889041247964, 7822645058523022688, 7176964512877741619, 2086571625238332354, 8242659079974169821, 1414172889041247964, 4955708925362059469, 7176964512877741619, 2086571625238332354, 8242659079974169821, 4977592835381219096, 8009690529124638334, 717696451287

In [7]:
def encrypt_data_with_aes(data, key):
    encrypted_data = "".join(chr(ord(c) ^ key) for c in data)
    return encrypted_data

In [8]:
def encrypt_aes_key_with_rsa(aes_key, public_key):
    e, n = public_key
    return power(aes_key, e, n)

In [9]:
def encrypt_csv_file(input_filename, public_key):
    aes_key = random.randint(1, 255) 

    with open(input_filename, 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        data = "\n".join([",".join(row) for row in reader])

    encrypted_data = encrypt_data_with_aes(data, aes_key)
    encrypted_aes_key = encrypt_aes_key_with_rsa(aes_key, public_key)

    with open("23_(SYS)_CC_Transmitted_User_Output_Data.csv", 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow([encrypted_aes_key])
        writer.writerow([encrypted_data])

    return encrypted_aes_key, encrypted_data

In [10]:
encrypted_aes_key, encrypted_data = encrypt_csv_file(input_filename, sys_rsa_2_public_key)
print("File encrypted successfully. Data saved in '22_(SYS)_CC_Predicted_User_Output_Data.csv'.")

print(encrypted_aes_key, encrypted_data, len(encrypted_data))

File encrypted successfully. Data saved in '22_(SYS)_CC_Predicted_User_Output_Data.csv'.
4057044880797286020 ª¶£¯´¢¬¶®´«®¶ª´®¨¶ª´¨¨¶«´ª¨¶ª´¯¨¶ª´¨£¶ª´¨«¶ª´©¯¶ª´¨£¶ª´¨©¶ª´©£¶«´«« 67


In [11]:
sys_rsa_1_private_key = load_key("16_SYS_RSA_1_Private_Key.pkl")
sys_rsa_1_private_key

(859385927303833, 14124542637770933)

In [12]:
sys_rsa_1_public_key = load_key("16_SYS_RSA_1_Public_Key.pkl")
sys_rsa_1_public_key

(1943022477995605, 14124542637770933)

In [13]:
def split_message(message, max_size):
    encoded = message.encode()
    chunks = [encoded[i:i + max_size] for i in range(0, len(encoded), max_size)]
    return [int.from_bytes(chunk, 'big') for chunk in chunks]

In [14]:
def sign_message(message, private_key, n):
    d, n = private_key
    max_bytes = (n.bit_length() // 8) - 1 
    message_chunks = split_message(message, max_bytes)
    
    signatures = [power(chunk, d, n) for chunk in message_chunks] 
    return signatures

In [15]:
signatures = sign_message(str(encrypted_aes_key) + '\n' + encrypted_data, sys_rsa_1_private_key, sys_rsa_1_public_key[1])
signatures

[2912602002030553,
 1821728777851964,
 9203313018508561,
 11831230424735924,
 12094984281175484,
 13842526337431544,
 3751300258767807,
 12076399661516534,
 11017051521045065,
 1825770349779046,
 8975767204885959,
 1825770349779046,
 1239795100165385,
 2868922631205115,
 4988525001699287,
 1825770349779046,
 11491710719024632,
 1825770349779046,
 11719988329442486,
 1825770349779046,
 8929700408445117,
 1825770349779046,
 11675006391287260,
 1825770349779046,
 11719988329442486,
 1825770349779046,
 13517465441076400,
 1825770349779046,
 8774550255828162,
 2868922631205115,
 10106996148687205]

In [16]:
with open("23_(SYS)_CC_Transmitted_User_Output_Data.csv", 'a', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow([signatures])